# Preprocessing OptiTrack Data
This script reads the exported CSV takes and transform the marker coordinates into the coordinate system of the full-touch smartphone. In case an exported version already exists, the take is skipped.

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from datetime import datetime
import time
import os
from IPython import display

%matplotlib inline

%run py/constants.py
%run py/take_processing_methods.py
%run py/marker_processing_methods.py
%run py/heatmap_methods.py
%run py/grasp_property_methods.py
%run py/rasterization_methods.py
%run py/labeling_names.py

In [ ]:
def read_take(take_file):
    """
    Reads the given take file and returns it in pandas format. 
    RETURN: Pandas dataframe, recording start as datetime
    """
    f_csv = open(take_file, 'r')
    marker_names = []   
    coordinates = [] 
    marker_type = []
    time_string = ""

    for i in range(0, 7):
        line = f_csv.readline()

        if i == 0:
            time_string = line.split(',')[9]
        if i == 3:
            marker_names = line.split(",")
        if i == 5:
            marker_type = line.split(",")
        if i == 6:
            coordinates = line.split(",")
    
    # Create the list of unique marker names to pass to the read_csv function.
    for i in range(0, len(marker_names)):
        newname = None
        for jname in joints:
            if jname.lower() in marker_names[i].lower() and not ("fka" in marker_names[i].lower()):
                newname = jname + "_" + str(coordinates[i])
                marker_names[i] = newname.strip()
        
        if newname == None:
            newname = marker_names[i] + "_" + str(coordinates[i])
            marker_names[i] = newname.strip()
            
        if marker_type[i].strip() != "Position":
            marker_names[i] = marker_names[i] + "_" + marker_type[i]
                
    marker_names[0] = 'Frame'
    marker_names[1] = 'Time'
    f_csv.close()
    
    # Importing one take (will be all takes later on)
    df = pd.read_csv(take_file, skiprows=list(range(0,7)), names=marker_names)
    df = df.set_index(df.Frame, drop=False)
    
    # retrieve datetime and fix 12AM error of OptiTrack Motive
    dt = datetime.strptime(time_string, '%Y-%m-%d %I.%M.%S.%f %p')
    if dt.hour == 0 and dt.strftime("%p") == "AM":
        dt = dt.replace(hour=12)
    
    return df, dt

In [ ]:
def get_transformed_marker(frame_index, marker_id, df, rot_matrices):
    rot_matrix = rot_matrices[df.index.get_loc(frame_index)].T

    rb_pivot =[df[df.columns[6]].ix[frame_index], df[df.columns[7]].ix[frame_index], df[df.columns[8]].ix[frame_index]]

    marker = [df[marker_id + "_X"].ix[frame_index], df[marker_id + "_Y"].ix[frame_index], df[marker_id + "_Z"].ix[frame_index]]  
    marker = inv_trans(marker, rb_pivot, rot_matrix)

    return marker

In [ ]:
def quaternion_matrix(row):
    """
    Converts a quaternion to a rotation matrix.
    Source: http://fabiensanglard.net/doom3_documentation/37726-293748.pdf
    RETURN: Rotation matrix as numpy array.
    """
    x = row[rigid_body_name + '_X_Rotation']
    y = row[rigid_body_name + '_Y_Rotation']
    z = row[rigid_body_name + '_Z_Rotation']
    w = row[rigid_body_name + '_W_Rotation']
    
    rot_matrix = [
        1.0-2*y*y-2*z*z, 2.0*x*y+2*w*z, 2.0*x*z - 2*w*y,
        2.0*x*y - 2*w*z, 1.0-2*x*x-2*z*z, 2.0*y*z+2*w*x,
        2.0*x*z+2*w*y, 2.0*y*z-2*w*x, 1.0-2*x*x-2*y*y]
    
    return rot_matrix

In [ ]:
def inv_trans(p, pos, rot):
    """
    Transform a marker into the rigid body coordinate system
    """
    p = np.subtract(p, pos)
    p = np.dot(p, rot.T)
    p = [p[0], p[2], p[1]]
    return p

In [ ]:
def trans(prow):
    markers = ["Thumb_Fn", "Thumb_DIP", "Thumb_PIP", "Index_Fn", "Index_DIP", "Index_PIP", "Middle_Fn", "Middle_DIP", "Middle_PIP", "Ring_Fn", "Ring_DIP", "Ring_PIP", "Little_Fn", "Little_DIP", "Little_PIP"]
    row = prow.copy(deep=True)
    
    # iterate over all rows
    for m in markers:
        x = row[m + "_X"]
        y = row[m + "_Y"]
        z = row[m + "_Z"]
        
        pivot = row['Pivot']
        rot_matrix = row['RotationMatrix']
        
        if not math.isnan(x):
            marker = [x, y, z]
            marker = inv_trans(marker, pivot, rot_matrix)
            
            row[m + "_X"] = marker[0]
            row[m + "_Y"] = marker[1]
            row[m + "_Z"] = marker[2]
            
    return row

In [ ]:
def log(s):
    with open("status_PY01_preprocessing.txt", "a") as myfile:
        myfile.write("[" + str(datetime.now()) + "] " + s + "\n")
    print("[" + str(datetime.now()) + "] " + s)

In [ ]:
for filename in os.listdir("./data/optitrack/"):
    if filename.endswith(".csv"):
        current_participant = filename.split(".")[0]
        rigid_body_name = RIGID_BODY_NAMES[current_participant]
        
        if os.path.isfile("./data/transformed_optitrack/dfTransformed_" + str(current_participant) + ".pkl") :
            log(current_participant + " is already available. Skipped.")
            continue
        
        log("Reading " + current_participant)
        df, dt = read_take("./data/optitrack/" + current_participant + ".csv")
        df['Time'] = ((df['Time'] + dt.timestamp()) * 1000)
        
        x = df[df.columns[2]]
        y = df[df.columns[3]]
        z = df[df.columns[4]]
        w = df[df.columns[5]]

        rot_matrix = np.array([
            [1-2*y*y-2*z*z, 2*x*y+2*w*z, 2*x*z - 2*w*y],
            [2*x*y - 2*w*z, 1-2*x*x-2*z*z, 2*y*z+2*w*x],
            [2*x*z+2*w*y, 2*y*z-2*w*x, 1-2*x*x-2*y*y]])

        angle = np.degrees(np.arccos(np.dot((rot_matrix[:,1,:].T), [0, 1, 0])))
        adf = pd.DataFrame(angle)
        df2 = adf[np.logical_not(adf[0].isnull())]
        df3 = df2[df2[0] < df2[0].mean() + df2[0].std()*2]
        dfFinal = df.iloc[df3.index]
        
        log("Creating RotationMatrix")
        dfFinal['RotationMatrix'] = dfFinal.apply(lambda row : quaternion_matrix(row), axis=1)
        dfFinal.RotationMatrix = dfFinal.RotationMatrix.apply(lambda x : np.array(x).reshape(3,3))
        dfFinal['Pivot'] = dfFinal.apply(lambda row : [row[rigid_body_name + '_X'], row[rigid_body_name + '_Y'], row[rigid_body_name + '_Z']], axis=1)
        dfFinal['Pivot'] = dfFinal.Pivot.apply(lambda x : np.array(x))
        
        log("Applying transformation")
        dfTransformed = dfFinal.apply(lambda x : trans(x), axis=1)
        dfTransformed.to_pickle("./data/transformed_optitrack/dfTransformed_" + current_participant + ".pkl")

[2018-02-22 18:07:38.358174] P8 is already available. Skipped.
[2018-02-22 18:07:38.359182] P19 is already available. Skipped.
[2018-02-22 18:07:38.359572] P14 is already available. Skipped.
[2018-02-22 18:07:38.359886] P16 is already available. Skipped.
[2018-02-22 18:07:38.360196] P22 is already available. Skipped.
[2018-02-22 18:07:38.360504] P4 is already available. Skipped.
[2018-02-22 18:07:38.360806] P13 is already available. Skipped.
[2018-02-22 18:07:38.361124] Reading P18
[2018-02-22 18:08:03.493666] Creating RotationMatrix
[2018-02-22 18:10:44.385126] Applying transformation
[2018-02-22 18:48:57.225237] P6 is already available. Skipped.
[2018-02-22 18:48:57.230328] Reading P9
[2018-02-22 18:49:19.178777] Creating RotationMatrix
[2018-02-22 18:51:44.342974] Applying transformation
